In [1]:
#!/usr/bin/env python3
"""Visualization script for Go1 with height scanner."""
import os
import subprocess
# Tell XLA to use Triton GEMM
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags
os.environ['MUJOCO_GL'] = 'egl'

import jax
import jax.numpy as jp
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.agents.ppo import losses as ppo_losses

import mujoco
from mujoco_playground import wrapper
from mujoco_playground import registry
from mujoco_playground.config import locomotion_params
from custom_env import Joystick, default_config

from datetime import datetime
import mediapy as media
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import functools
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as IPyimage, display, HTML, clear_output

from utils import render_video_during_training, evaluate_policy

scene_option = mujoco.MjvOption()
scene_option.geomgroup[2] = True   # Show visual geoms
scene_option.geomgroup[3] = False  # Hide collision geoms
scene_option.geomgroup[5] = True   # Show sites (including height scanner visualization)
scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True  # Show contact points
scene_option.flags[mujoco.mjtVisFlag.mjVIS_RANGEFINDER] = True
print("Creating Visualization...")

xml_path = 'custom_env.xml' # 'custom_env_debug_wall.xml'
env = Joystick(xml_path=xml_path, config=default_config())

# JIT compile the functions for speed
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)
jit_terrain_height = jax.jit(env._get_torso_terrain_height)

seed = 1234
num_envs = ()
key = jax.random.PRNGKey(seed)
key, key_env, eval_key, key_policy, key_value = jax.random.split(key, 5)
key_envs = jax.random.split(key_env, num_envs)
env_state = jit_reset(key_envs)

Creating Visualization...


ERROR:2025-10-01 10:27:09,317:jax._src.xla_bridge:444: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/loke/miniconda3/envs/FinalRL/lib/python3.10/site-packages/jax/_src/xla_bridge.py", line 442, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/home/loke/miniconda3/envs/FinalRL/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 324, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/home/loke/miniconda3/envs/FinalRL/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 281, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: Unknown CUDA error 303; cuGetErrorName failed. This probably means that JAX was unable to load the CUDA libraries.


# Teacher Policy

- pretrained inside train.ipynb
- we want to load the parameters

- Inputs: privileged_state with heightmap
- Output: action

### Load parameters for pre-trained teacher


In [ ]:
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.agents.ppo import losses as ppo_losses
from brax.training.acme import running_statistics


# Needs to match training
obs_shape = (96,)
action_size = env.action_size

# Observation normalisation
loaded_params = np.load("params.npy", allow_pickle=True)
normalizer_params = loaded_params[0]  # The first element should be the RunningStatisticsState
normalize = running_statistics.normalize

# Setup
ppo_params = locomotion_params.brax_ppo_config('Go1JoystickRoughTerrain')
network_factory = ppo_networks.make_ppo_networks
if "network_factory" in ppo_params:
    network_factory = functools.partial(
        ppo_networks.make_ppo_networks,
        **ppo_params.network_factory
    )

ppo_network = network_factory(
    obs_shape, action_size, preprocess_observations_fn=normalize
)

init_params = ppo_losses.PPONetworkParams(
    policy=ppo_network.policy_network.init(key_policy),
    value=ppo_network.value_network.init(key_value),
)

# Create policy function
make_policy = ppo_networks.make_inference_fn(ppo_network)

params = np.load("params.npy", allow_pickle=True)

jit_inference_fn = jax.jit(make_policy(params, deterministic=True))

IndexError: list index out of range

# Student Policy

Experiment 1: training with a newly initialized LSTM for a Recurrent Neural Network


In [24]:
# Policy Definition
import jax.numpy as jnp
import jax
import flax.linen as nn

lstm = nn.RNN(nn.LSTMCell(features=64))
x = jnp.ones((10, 50, 52)) # (batch, time, features)
variables = lstm.init(jax.random.PRNGKey(0), x)


# Training


In [25]:
from custom_ppo_train import _maybe_wrap_env

seed = 1234

episodes = 1
envs_per_episode = 1
episode_length = 1000
action_repeat = 1

learning_rate = 1e-5


## Training Loop

In [29]:
from mujoco_playground._src.gait import draw_joystick_command
env_cfg = default_config()
env_cfg.pert_config.enable = True
env_cfg.pert_config.velocity_kick = [0.0, 0.0]
env_cfg.pert_config.kick_wait_times = [5.0, 15.0]
env_cfg.command_config.a = [1.5, 0.8, 2*jp.pi]


# Loop over "runs"
for episode in range(episodes):
    key = jax.random.PRNGKey(seed)
    key, key_env, eval_key, key_policy, key_value = jax.random.split(key, 5)

    wrapper_env = _maybe_wrap_env(
        env,
        wrap_env=True,
        num_envs=envs_per_episode,
        episode_length=episode_length,
        action_repeat=action_repeat,
        key_env=key_env,
    )

    # Reset environment 
    reset_fn = jax.jit(env.reset)
    key_envs = jax.random.split(key_env, num_envs)
    env_state = reset_fn(key_envs)
    # print(f"ENV STATE: {env_state.done}")
    obs_shape = jax.tree_util.tree_map(lambda x: x.shape[1:], env_state.obs)
    # print(f"OBS STATE: {env_state.obs}")


    rng = jax.random.PRNGKey(0)
    raw_command = jax.random.uniform(rng, shape=(3), minval=0.0, maxval=1.0)
    raw_command = jp.array([0.5, 0.0, 0.0]) 

    command = jp.array([
    raw_command[0] * env_cfg.command_config.a[0], 
    raw_command[1] * env_cfg.command_config.a[1],
    raw_command[2] * env_cfg.command_config.a[2] 
    ])
    state = jit_reset(rng)
    state.info["command"] = command

    # Visualisation storing
    rollout = []
    modify_scene_fns = []

    # Training data storing
    teacher_obs = []

    for _ in range(episode_length): # NOT DOING DONE: CHECK .done? - what does done even mean
        
        # Get new action
        act_rng, rng = jax.random.split(rng)
        ctrl, _ = jit_inference_fn(state.obs, act_rng) # we want to save ctrl?? 

        # Get new state based on action
        state = jit_step(state, ctrl)
        state.info["command"] = command

        
        # Visualisation
        rollout.append(state)
        xyz = np.array(state.data.xpos[env._torso_body_id])
        xyz += np.array([0, 0, 0.2])  # Offset for visualization
        x_axis = state.data.xmat[env._torso_body_id, 0]
        yaw = -np.arctan2(x_axis[1], x_axis[0])
        modify_scene_fns.append(
            functools.partial(
                draw_joystick_command,
                cmd=state.info["command"],
                xyz=xyz,
                theta=yaw,
                scl=abs(state.info["command"][0]) / env_cfg.command_config.a[0],
            )
        )

    
    # Rendering config
    render_every = 2  # Frame skip for smoother visualization
    fps = 1.0 / env.dt / render_every
    traj = rollout[::render_every]
    mod_fns = modify_scene_fns[::render_every]

    scene_option = mujoco.MjvOption()
    scene_option.geomgroup[2] = True  # Show visual geoms
    scene_option.geomgroup[3] = False  # Hide collision geoms
    scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
    scene_option.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = False
    scene_option.flags[mujoco.mjtVisFlag.mjVIS_PERTFORCE] = True

    # Render
    frames = env.render(
    traj,
    camera="track",
    scene_option=scene_option,
    width=640,
    height=480,
    modify_scene_fns=mod_fns,
    )   
    media.show_video(frames, fps=fps)





    # if normalize_observations:
    #     normalize = running_statistics.normalize

    # optimizer = optax.adam(learning_rate=learning_rate)

    # loss_fn = #

    # gradient_update_fn = #

100%|██████████| 500/500 [00:05<00:00, 92.56it/s] 


This browser does not support the video tag.


In [ ]:
done = False
    
while not done:
    act_rng, rng = jax.random.split(rng)
    ctrl, _ = jit_inference_fn(state.obs, act_rng)
    done = True